<a href="https://colab.research.google.com/github/RafsanJany-44/ARC/blob/master/Youtube_Statistics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pafy
!pip install youtube-dl==2020.12.2
!pip install pytube

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 33.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 KB 4.7 MB/s eta 0:00:00


In [ ]:
import pafy
def janina(url):
  video = pafy.new(url) 
    
  # getting view count of video
  views = video.viewcount
  length = video.length
  likes = video.likes
  dislikes = video.dislikes
  duration = video.duration
  title = video.title
  author = video.author
    
  # printing the value
  dic={
  "author" : str(author),
  "title" : str(title),
  "View Count": int(views),
  "length" : (length),
  "likes" : str(likes),
  "dislikes" : str(dislikes),
  "duration" : str(duration)
  }

  return dic

In [ ]:
janina("https://www.youtube.com/watch?v=yb2MstCVGQY")

{'author': 'GetsetflySCIENCE',
 'title': "2060 को दुनिया ख़त्म होने वाली है - NEWTON की भविष्यवाणी | Newton's Prediction for The Future",
 'View Count': 722980,
 'length': 613,
 'likes': '47636',
 'dislikes': 'None',
 'duration': '00:10:13'}

##for a play list

In [ ]:
from pytube import Playlist

p = Playlist("https://www.youtube.com/playlist?list=PLLssT5z_DsK-h9vYZkQkYNWcItqhlRJLN")

for i in p:
  print(janina(i))

#getting comments

In [ ]:
from googleapiclient.discovery import build
import json
import requests
import time
import pandas as pd
import re
import numpy as np

## Taking input and Splitting the video Id

In [ ]:
url_input = "https://www.youtube.com/watch?v=yb2MstCVGQY"
video_id_split =url_input.split('=')
video_id=video_id_split[1]
print(video_id)

yb2MstCVGQY


## Total number of comments

In [ ]:

api_key = ''
url = 'https://www.googleapis.com/youtube/v3/videos?id='+video_id+'&key='+api_key+'&part=id,statistics'
response_info=requests.get(url).json()

# Contains all comments (both top-level comments and replies to those comments).
all_comments = []

# Get the total (sum) of comments the video has: 
for comment_count in response_info['items']:
  total = int(comment_count['statistics']['commentCount'])

# Show the total amount of comments for the given video: 
print("Total comments for (" + video_id + "): " + str(total))


Total comments for (yb2MstCVGQY): 1577


## Storing all the comments in a list


In [ ]:
def getAllTopLevelCommentReplies(topCommentId, token): 
  """
  Recursive function that retrieves the replies a given comment has.
  """

  replies_response=youtube.comments().list(part='snippet',maxResults=100,parentId=topCommentId,pageToken=token).execute()

  for indx, reply in enumerate(replies_response['items']):
    all_comments.append(reply['snippet']['textDisplay'])
    
  if "nextPageToken" in replies_response: 
    return getAllTopLevelCommentReplies(topCommentId, replies_response['nextPageToken'])
  else:
    return []
      
def get_comments(youtube, video_id, token): 
  """
  Recursive function that retrieves the comments (top-level ones) a given video has.
  """

  global all_comments
  totalReplyCount = 0
  token_reply = None

  if (len(token.strip()) == 0): 
    all_comments = []

  if (token == ''): 
    video_response=youtube.commentThreads().list(part='snippet',maxResults=100,videoId=video_id,order='relevance').execute() 
  else: 
    video_response=youtube.commentThreads().list(part='snippet',maxResults=100,videoId=video_id,order='relevance',pageToken=token).execute() 
  
  ## ---- for indx, item in enumerate(video_response['items']):
  ## ----   all_comments.append(item['snippet']['topLevelComment']['snippet']['textDisplay'])
  ## ----   totalReplyCount = item['snippet']['totalReplyCount']
  ## ----   if (totalReplyCount > 0): 
  ## ----     all_comments.append(getAllTopLevelCommentReplies(item['id'], None)) 
  ## ----     
  ## ---- if "nextPageToken" in video_response: 
  ## ----   return get_comments(youtube, video_id, video_response['nextPageToken'])
  ## ---- else: 
  ## ----   # Remove empty elements added to the list "due to the return in both functions":
  ## ----   all_comments = [x for x in all_comments if len(x) > 0]
  ## ----   return []

  # INICIO 

  # Loop comments from the video: 
  for indx, item in enumerate(video_response['items']): 
    # Append coments:
    all_comments.append("COMMENT WITH " + str(item['snippet']['totalReplyCount']) + " replies: " + item['snippet']['topLevelComment']['snippet']['textDisplay'])

    # Get total reply count: 
    totalReplyCount = item['snippet']['totalReplyCount']

    # If the comment has replies, get them:
    if (totalReplyCount > 0): 
      # Get replies - first batch: 
      replies_response=youtube.comments().list(part='snippet',maxResults=100,parentId=item['id']).execute()
      for indx, reply in enumerate(replies_response['items']):
        # Append the replies to the main array: 
        all_comments.append((" "*2) + "=>FIRST CALLBACK REPLY: " + reply['snippet']['textDisplay'])

      # If the reply has a token for get more replies, loop those replies 
      # and add those replies to the main array: 
      while "nextPageToken" in replies_response:
        token_reply = replies_response['nextPageToken']
        replies_response=youtube.comments().list(part='snippet',maxResults=100,parentId=item['id'],pageToken=token_reply).execute()
        for indx, reply in enumerate(replies_response['items']):
          all_comments.append((" "*4) + "==>WHILE GETTING REPLIES: " + reply['snippet']['textDisplay'])
    
  # Check if the video_response has more comments:
  if "nextPageToken" in video_response: 
    return get_comments(youtube, video_id, video_response['nextPageToken']) 
  else: 
    # Remove empty elements added to the list "due to the return in both functions":
    all_comments = [x for x in all_comments if len(x) > 0]
    print("Fin")
    return []
  
  # FIN

  ## 28/04/2022: Reviewed again and here are my results: 
  ## Both codes works fine = problem is: youtube does not match the count shown 
  ## in the website -comment section- and youtube data api. some comments *might* 
  ## be under approval or youtube just removes without updating the "totalReplyCount" value.
  ## also, commment replies are not in date order, those are retrieved in descendent order 
  ## (newest replies are first in the results).
      
all_comments=[]
qtyReplies = 0
qtyMainComments = 0

youtube = build('youtube', 'v3',developerKey=api_key)
comments = get_comments(youtube,video_id,'')

# Show results: 
print("All total comments obtained: "  + str(len(all_comments)))

Fin
All total comments obtained: 1499


In [ ]:
all_comments

['COMMENT WITH 16 replies: For Binomo Education, Click <a href="http://binomoindia.in/GETSETFLY_SCIENCE">http://binomoindia.in/GETSETFLY_SCIENCE</a> \r<br>and get Rs.65,000 for Binomo tutorial.\r<br>Use ADV107 promo code for +100% on the first deposit. Thank me later!  \r<br>Risk Warning! Your capital might be at risk!\r<br>Go subscribe to Binomo&#39;s Official Instagram Account - <a href="https://www.instagram.com/binomo/">https://www.instagram.com/binomo/</a>',
 '  =>FIRST CALLBACK REPLY: Njj',
 '  =>FIRST CALLBACK REPLY: Ho jaye jaldi hi khatam yahi acha rahe ga😁',
 '  =>FIRST CALLBACK REPLY: Y',
 '  =>FIRST CALLBACK REPLY: AND well these books are given by the Truth Allah',
 '  =>FIRST CALLBACK REPLY: What are the 4 religious books?\r<br>ANS: Being Muslims, we are familiar to four Revealed books: i) The Taurah; ii) Psalms of David; iii) The Bible &amp; iv) The Holy Quran. These four mentioned books were revealed to four different prophets. Taurah was revealed to Moses (PBUH); Psalm

## Converting the list to dataframe


In [ ]:
df = pd.DataFrame(all_comments,columns=['Comments'])
print(df)

                                               Comments
0     COMMENT WITH 16 replies: For Binomo Education,...
1                           =>FIRST CALLBACK REPLY: Njj
2       =>FIRST CALLBACK REPLY: Ho jaye jaldi hi kha...
3                             =>FIRST CALLBACK REPLY: Y
4       =>FIRST CALLBACK REPLY: AND well these books...
...                                                 ...
1494                      =>FIRST CALLBACK REPLY: Burrr
1495                  =>FIRST CALLBACK REPLY: Op boltat
1496                        =>FIRST CALLBACK REPLY: Wow
1497                        =>FIRST CALLBACK REPLY: Hmm
1498                        =>FIRST CALLBACK REPLY: Hem

[1499 rows x 1 columns]


# Data Cleaning


## Removing all the emoji's from the dataframe


In [ ]:
df = df.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))
df

,Comments
0,"COMMENT WITH 16 replies: For Binomo Education,..."
1,=>FIRST CALLBACK REPLY: Njj
2,=>FIRST CALLBACK REPLY: Ho jaye jaldi hi kha...
3,=>FIRST CALLBACK REPLY: Y
4,=>FIRST CALLBACK REPLY: AND well these books...
...,...
1494,=>FIRST CALLBACK REPLY: Burrr
1495,=>FIRST CALLBACK REPLY: Op boltat
1496,=>FIRST CALLBACK REPLY: Wow
1497,=>FIRST CALLBACK REPLY: Hmm


## Removing all the url's from the data frame

In [ ]:
df['Comments'] = df['Comments'].apply(lambda x: re.split('<a href="https:\/\/.*', str(x))[0])
df

,Comments
0,4m soon
1,
2,Himachali are more savage then mumbai and delh...
3,10:5 the car is onn
4,"my non-delhi, non-mumbai ass watching this:"
...,...
3348,Epic
3349,waha tanmay .......tm toh smart ho
3350,second
3351,First


## Removing all special characters

In [ ]:
def process_content(content):
    return " ".join(re.findall("[A-Za-z]+",content))

df['Comments'] = df['Comments'].apply(process_content)
df

,Comments
0,m soon
1,
2,Himachali are more savage then mumbai and delh...
3,the car is onn
4,my non delhi non mumbai ass watching this
...,...
3348,Epic
3349,waha tanmay tm toh smart ho
3350,second
3351,First


## Converting to lower case

In [ ]:
df['Comments'] = df['Comments'].str.lower()
df

,Comments
0,m soon
1,
2,himachali are more savage then mumbai and delh...
3,the car is onn
4,my non delhi non mumbai ass watching this
...,...
3348,epic
3349,waha tanmay tm toh smart ho
3350,second
3351,first


## Removing empty rows

In [ ]:
df['Comments'].replace('', np.nan, inplace=True)
df.dropna()

,Comments
0,m soon
2,himachali are more savage then mumbai and delh...
3,the car is onn
4,my non delhi non mumbai ass watching this
5,after seeing this br literally nobody br girls...
...,...
3348,epic
3349,waha tanmay tm toh smart ho
3350,second
3351,first
